In [52]:
import os

import numpy as np
from clean import data_dir

os.environ["KERAS_BACKEND"] = "jax"
import keras
import keras_tuner
from keras import layers

In [27]:
splits = np.load(data_dir("samples/train_joined_not_interpolate_splits.npz"))
i_train, i_tune, i_test, t_train, t_tune, t_test = (
    splits[part]
    for part in ["i_train", "i_tune", "i_test", "t_train", "t_tune", "t_test"]
)

In [29]:
# Functions for the model search:
input_norm = layers.Normalization(
    axis=-1, mean=i_train.mean(axis=0), variance=i_train.var(axis=0)
)

output_denorm = layers.Normalization(
    axis=-1,
    invert=True,
    mean=t_train.mean(axis=0),
    variance=t_train.var(axis=0),
)

In [44]:
def directly_weighted_mse(y_true, y_pred):
    return keras.ops.mean(((y_true - y_pred) * (y_true / t_train.mean())) ** 2)

In [45]:
def build_model(hp: keras_tuner.HyperParameters):
    model = keras.Sequential(
        [layers.Input((12,), batch_size=50, name="input")], name="prediction_model"
    )

    if hp.Boolean("normalize_input"):
        model.add(input_norm)

    for i in range(hp.Int("num_layers", 1, 5, step=1)):
        model.add(
            layers.Dense(
                hp.Int(f"units_{i}", 10, 100, step=10),
                activation=hp.Choice("activation_type", ["relu", "tanh"]),
                name=f"layer_{i}",
            )
        )

        if hp.Boolean("dropout"):
            model.add(layers.Dropout(hp.Float("dropout_rate", 0.05, 0.35, step=0.1)))

    model.add(
        layers.Dense(1, name="output"),
    )

    if hp.Boolean("denormalize_output"):
        model.add(output_denorm)

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",  # or: directly_weighted_mse
        metrics=["mape", "mse"],
    )
    return model

In [46]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=1,
    overwrite=False,
    seed=1,
    directory="keras_checkpoints",
    project_name="time_nn",
)

Reloading Tuner from keras_checkpoints/time_nn/tuner0.json


In [47]:
fit_args = dict(
    x=i_train,
    y=t_train,
    validation_data=(i_tune, t_tune),
    epochs=10,
    callbacks=[
        keras.callbacks.EarlyStopping(
            patience=1, restore_best_weights=True, monitor="val_loss", mode="min"
        )
    ],
    batch_size=50_000,
    verbose=False,
)

In [48]:
tuner.search(**fit_args)

In [13]:
model = build_model(tuner.get_best_hyperparameters()[0])

history = model.fit(**fit_args)
model.save("../models/model_in_time.keras")

In [ ]:
# keras.utils.plot_model(
#     model, show_shapes=True, show_layer_activations=True, to_file="time.png"
# )

In [20]:
np.save(data_dir("pred/time_mse.npy"), model(i_test))
print("RMSE:", np.sqrt(model.evaluate(x=i_test, y=t_test, batch_size=50_000)[0]))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 439.1220 - mean_absolute_percentage_error: 915458.6875
RMSE: 20.973337595446704


In [51]:
# Variant with directly weighted training loss
np.save(data_dir("pred/time_wgh.npy"), model(i_test))
print("RMSE:", np.sqrt(model.evaluate(x=i_test, y=t_test, batch_size=50_000)[2]))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1776.1830 - mape: 4725832.0000 - mse: 2301.8616
RMSE: 47.97337111291534
